In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#pip uninstall tensorflow
#pip uninstall keras

In [ ]:
#!pip install tensorflow==2.6.0

In [ ]:
#pip install keras==2.6.0

In [6]:
import wget
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
sample_data_path = "/code/"

In [9]:
!unzip "/content/drive/MyDrive/Capstone Project/Code/newsqa_gl"

Archive:  /content/drive/MyDrive/Capstone Project/Code/newsqa_gl.zip
   creating: newsqa_gl/.ipynb_checkpoints/
  inflating: newsqa_gl/.ipynb_checkpoints/Untitled1-checkpoint.ipynb  
  inflating: newsqa_gl/.ipynb_checkpoints/Untitled-checkpoint.ipynb  
 extracting: newsqa_gl/__.init__.py  
   creating: newsqa_gl/code/
 extracting: newsqa_gl/code/__.init__.py  
   creating: newsqa_gl/code/models/
 extracting: newsqa_gl/code/models/__.init__.py  
  inflating: newsqa_gl/code/models/custom_model.py  
   creating: newsqa_gl/code/prep/
 extracting: newsqa_gl/code/prep/__.init__.py  
  inflating: newsqa_gl/code/prep/load_data.py  
   creating: newsqa_gl/code/utils/
 extracting: newsqa_gl/code/utils/__.init__.py  
  inflating: newsqa_gl/code/utils/config.py  
  inflating: newsqa_gl/code/utils/utils.py  
   creating: newsqa_gl/data/
  inflating: newsqa_gl/data/combined-newsqa-data-v1.json  
   creating: newsqa_gl/data/glove.6B.200d.txt/
  inflating: newsqa_gl/data/glove.6B.200d.txt/glove.6B.200

In [5]:
%pip install tensorflow_hub

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import tokenization
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import concatenate
import pandas as pd
import numpy as np

In [6]:
print(tf.__version__)
print(tf.keras.__version__)

2.7.0
2.7.0


In [45]:
module_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(module_url, trainable=True)

In [2]:
data = pd.read_csv('code/sample_data.csv')

NameError: name 'pd' is not defined

In [11]:
data.head()

,story_id,Question,paragraph,ans_st_index,ans_en_index,Question1
0,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,What was the amount of children murdered?,"NEW DELHI, India (CNN) -- A high court in nort...",294.0,297.0,what was the amount of children murdered
1,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,When was Pandher sentenced to death?,"NEW DELHI, India (CNN) -- A high court in nort...",261.0,271.0,when was pandher sentenced to death
2,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,The court aquitted Moninder Singh Pandher of w...,"NEW DELHI, India (CNN) -- A high court in nort...",624.0,640.0,the court aquitted moninder singh pandher of w...
3,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,who was acquitted,"NEW DELHI, India (CNN) -- A high court in nort...",195.0,218.0,who was acquitted
4,./cnn/stories/42d01e187213e86f5fe617fe32e716ff...,who was sentenced,"NEW DELHI, India (CNN) -- A high court in nort...",195.0,218.0,who was sentenced


In [33]:
p_sent=data.paragraph.tolist()
q_sent = data.Question1.tolist()
ans_st_index = data.ans_st_index.values

In [34]:
n_class = len(np.unique(ans_st_index))

In [35]:
lb = LabelEncoder()
y=lb.fit_transform(ans_st_index)
#y = to_categorical(y,num_classes=n_class)

In [36]:
vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [37]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [38]:
def build_model(bert_layer, max_len=512):
    
    q_input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="q_input_word_ids")
    q_input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="q_input_mask")
    q_segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="q_segment_ids")
    
    pooled_output, sequence_output = bert_layer([q_input_ids, q_input_mask, q_segment_ids])
    clf_output = sequence_output[:, 0, :]
    net_q = tf.keras.layers.Dense(64, activation='relu')(clf_output)

    p_input_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="p_input_word_ids")
    p_input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="p_input_mask")
    p_segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="p_segment_ids")
    
    pooled_output, sequence_output = bert_layer([p_input_ids, p_input_mask, p_segment_ids])
    clf_output = sequence_output[:, 0, :]
    net_p = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    
    net_p = tf.keras.layers.Dropout(0.2)(net_p)

    net = concatenate([net_q, net_p])
    
    net = tf.keras.layers.Dropout(0.2)(net)
    net = tf.keras.layers.Dense(32, activation='relu')(net)
    net = tf.keras.layers.Dropout(0.2)(net)
    out = tf.keras.layers.Dense(n_class, activation='softmax')(net)
    
    model = tf.keras.models.Model(inputs=[[q_input_ids, q_input_mask, q_segment_ids]
                                          ,[p_input_ids, p_input_mask, p_segment_ids]]
                                  , outputs=out)
    model.compile(tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
    return(model)

In [46]:
max_len = 150
train_input_p = bert_encode(p_sent, tokenizer, max_len=max_len)
train_input_q = bert_encode(q_sent, tokenizer, max_len=max_len)


In [47]:
train_labels = to_categorical(y,num_classes=n_class)
train_labels.shape

(201, 163)

In [48]:
model = build_model(bert_layer, max_len=max_len)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 q_input_word_ids (InputLayer)  [(None, 150)]        0           []                               
                                                                                                  
 q_input_mask (InputLayer)      [(None, 150)]        0           []                               
                                                                                                  
 q_segment_ids (InputLayer)     [(None, 150)]        0           []                               
                                                                                                  
 p_input_word_ids (InputLayer)  [(None, 150)]        0           []                               
                                                                                            

In [42]:
#checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)
#earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, verbose=1)

In [49]:
train_history = model.fit(
    [train_input_q,train_input_p], train_labels, 
    validation_split=0.2,
    epochs=16,
    callbacks=[],
    batch_size=20,
    verbose=1)

Epoch 1/16


ResourceExhaustedError: ignored